# Lecture 59: Activity recognition using 3D-CNN

#### Dataset: [UCF101](http://crcv.ucf.edu/data/UCF101.php)

In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from random import shuffle
from torchvision import transforms,datasets, models
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision
import os

import copy
import time

### Create train  and test list

In [ ]:
# Directory containing tensor of frames for each video
trainPath = 'ucf101_vidTensors/train/'
testPath = 'ucf101_vidTensors/test/'

In [ ]:
# Preparing train list
classes = os.listdir(trainPath)
classes.sort()
labels = np.arange(5)
trainShuffList = []
labelShuffList = []
for c in range(5):
    files = os.listdir(trainPath+classes[c])
    for f in files:
        trainShuffList.append(classes[c]+'/'+f)  
        labelShuffList.append(float(labels[c]))
trainList = list(zip(trainShuffList, labelShuffList))
shuffle(trainList)
trainShuffList, labelShuffList = zip(*trainList)

In [ ]:
# Preparing test list
testList = []
testLabelList = []
for c in range(5):
    files = os.listdir(testPath+classes[c])
    for f in files:
        testList.append(classes[c]+'/'+f)  
        testLabelList.append(float(labels[c]))

## Define network architecture

In [ ]:
class net_3DCNN(nn.Module):
    def __init__(self):
        super(net_3DCNN, self).__init__()   
        self.conv1 = nn.Conv3d(3, 16, kernel_size=5)   
        self.pool1 = nn.MaxPool3d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv3d(16, 32, kernel_size=3) 
        self.pool2 = nn.MaxPool3d(kernel_size=2,stride=2)
        self.conv3 = nn.Conv3d(32, 32, kernel_size=3) 
        self.pool3 = nn.AvgPool3d(kernel_size=4)   
        self.fc = nn.Linear(32*13*13,5)   
    
    def forward(self, x):        
        x = F.relu(self.conv1(x),inplace=True)
        x = self.pool1(x)
        x = F.relu(self.conv2(x),inplace=True)
        x = self.pool2(x)
        x = F.relu(self.conv3(x),inplace=True)
        x = self.pool3(x)        
        x = self.fc(x.view(x.size(0),-1))
        return x

## Define train routine

In [ ]:
def train(net, inputs, labels, optimizer, criterion):
    net.train(True)
    inputs, labels = Variable(inputs).float().cuda(), Variable(labels).cuda()    
    outputs = net(inputs)
#     print(outputs.size())
    _, predicted = torch.max(outputs.data, 1)     
    # Initialize gradients to zero
    optimizer.zero_grad() 
    # Compute loss/error
    loss = criterion(F.log_softmax(outputs), labels)
    # Backpropagate loss and compute gradients
    loss.backward()
    # Update the network parameters
    optimizer.step()
    correct = (predicted.cpu() == labels.data.cpu()).sum()
    return net, loss.data[0], correct    

## Define test routine

In [ ]:
def test(net, inputs, labels, criterion):
    net.train(False)
    inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
    outputs = net(inputs)
#     print(outputs.size())
    _, predicted = torch.max(outputs.data, 1)  
    # Compute loss/error
    loss = criterion(F.log_softmax(outputs), labels)   
    correct = (predicted.cpu() == labels.data.cpu()).sum()
    return loss.data[0], correct
    

### Initialize network

In [ ]:
net = net_3DCNN()
print(net)

In [ ]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')   
    net = net.cuda()

## Define loss function and optimizer

In [ ]:
criterion = nn.NLLLoss() # Negative Log-likelihood
optimizer = optim.Adam(net.parameters(), lr=1e-4) # Adam

## Train the network

In [ ]:
epochs = 10
bSize = 32
L = 32 # Depth/ no. of frames per video
bCount = len(trainShuffList)//bSize
lastBatch = len(trainShuffList)%bSize

test_bCount = len(testList)//bSize
test_lastBatch = len(testList)%bSize

trainLoss = []
trainAcc = []
testLoss = []
testAcc = []

for epochNum in range(epochs):
    trainList = list(zip(trainShuffList, labelShuffList))
    shuffle(trainList)
    trainShuffList, labelShuffList = zip(*trainList)
    trainRunLoss = 0.0
    testRunLoss = 0.0
    trainRunCorr = 0
    testRunCorr = 0
    epochStart = time.time()
    
    ## Train the network
    # Load data tensors batchwise     
    idx = 0    
    for bNum in range(bCount):
        first = True
        for dNum in range(idx,idx+bSize):
            if first:
                loadData = torch.load(trainPath+trainShuffList[dNum])
                sz = loadData.size(1)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                batchData = torch.index_select(loadData,dim=1,index=idx1).unsqueeze(0)
                batchLabel = torch.Tensor([labelShuffList[dNum]]).long()                          
                first = False                
            else:
                loadData = torch.load(trainPath+trainShuffList[dNum])
                sz = loadData.size(1)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                tempData = torch.index_select(loadData,dim=1,index=idx1).unsqueeze(0)
                batchData = torch.cat((batchData,tempData), dim=0)
                batchLabel = torch.cat((batchLabel,torch.Tensor([labelShuffList[dNum]]).long()),dim=0)            
        
        net, tr_loss, tr_corr = train(net, batchData, batchLabel, optimizer, criterion)
        trainRunLoss += tr_loss
        trainRunCorr += tr_corr
        idx += bSize
    if lastBatch != 0:        
        first = True
        for dNum in range(idx,idx+lastBatch):
            if first:
                loadData = torch.load(trainPath+trainShuffList[dNum])
                sz = loadData.size(1)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                batchData = torch.index_select(loadData,dim=1,index=idx1).unsqueeze(0)
                batchLabel = torch.Tensor([labelShuffList[dNum]]).long()
                first = False                
            else:
                loadData = torch.load(trainPath+trainShuffList[dNum])
                sz = loadData.size(1)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                tempData = torch.index_select(loadData,dim=1,index=idx1).unsqueeze(0)
                batchData = torch.cat((batchData,tempData), dim=0)
                batchLabel = torch.cat((batchLabel,torch.Tensor([labelShuffList[dNum]]).long()),dim=0)          
        net, tr_loss, tr_corr = train(net, batchData, batchLabel, optimizer, criterion)
        trainRunLoss += tr_loss
        trainRunCorr += tr_corr
    avgTrainLoss = trainRunLoss/float(len(trainShuffList))
    trainLoss.append(avgTrainLoss)
    avgTrainAcc = trainRunCorr/float(len(trainShuffList))
    trainAcc.append(avgTrainAcc)
    
    # Test the network
#     Load data tensors batchwise     
    idx = 0    
    for bNum in range(test_bCount):
        first = True
        for dNum in range(idx,idx+bSize): 
            if first:
                loadData = torch.load(testPath+testList[dNum])
                sz = loadData.size(1)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                batchData = torch.index_select(loadData,dim=1,index=idx1).unsqueeze(0)
                batchLabel = torch.Tensor([testLabelList[dNum]]).long()
                first = False                
            else:
                loadData = torch.load(testPath+testList[dNum])
                sz = loadData.size(1)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                tempData = torch.index_select(loadData,dim=1,index=idx1).unsqueeze(0)
                batchData = torch.cat((batchData,tempData), dim=0)
                batchLabel = torch.cat((batchLabel,torch.Tensor([testLabelList[dNum]]).long()),dim=0)            
        ts_loss, ts_corr = test(net, batchData, batchLabel, criterion)
        testRunLoss += ts_loss
        testRunCorr += ts_corr
        idx += bSize
    if test_lastBatch != 0:        
        first = True
        for dNum in range(idx,idx+test_lastBatch):
            if first:
                loadData = torch.load(testPath+testList[dNum])
                sz = loadData.size(1)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                batchData = torch.index_select(loadData,dim=1,index=idx1).unsqueeze(0)               
                batchLabel = torch.Tensor([testLabelList[dNum]]).long()
                first = False                
            else:
                loadData = torch.load(testPath+testList[dNum])
                sz = loadData.size(1)
                idx1 = torch.from_numpy(np.arange(0,(sz//L)*L,sz//L))
                tempData = torch.index_select(loadData,dim=1,index=idx1).unsqueeze(0)
                batchData = torch.cat((batchData,tempData), dim=0)
                batchLabel = torch.cat((batchLabel,torch.Tensor([testLabelList[dNum]]).long()),dim=0)          
        ts_loss, ts_corr = test(net, batchData, batchLabel, criterion)
        testRunLoss += ts_loss
        testRunCorr += tr_corr
    avgTestLoss = testRunLoss/float(len(testList))
    testLoss.append(avgTestLoss)
    avgTestAcc = testRunCorr/float(len(testList))
    testAcc.append(avgTestAcc)
    

    
    # Plotting training loss vs Epochs
    fig1 = plt.figure(1)        
    plt.plot(range(epochNum+1),trainLoss,'r-',label='train')  
    plt.plot(range(epochNum+1),testLoss,'g-',label='test') 
    if epochNum==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')   
    # Plotting testing accuracy vs Epochs
    fig2 = plt.figure(2)        
    plt.plot(range(epochNum+1),trainAcc,'r-',label='train')    
    plt.plot(range(epochNum+1),testAcc,'g-',label='test')        
    if epochNum==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
    
    epochEnd = time.time()-epochStart
    print('Iteration: {:.0f} /{:.0f};  Training Loss: {:.6f} ; Training Acc: {:.3f}'\
          .format(epochNum + 1,epochs, avgTrainLoss, avgTrainAcc*100))
    print('Iteration: {:.0f} /{:.0f};  Testing Loss: {:.6f} ; Testing Acc: {:.3f}'\
          .format(epochNum + 1,epochs, avgTestLoss, avgTestAcc*100))
    
    print('Time consumed: {:.0f}m {:.0f}s'.format(epochEnd//60,epochEnd%60))
        